# MCP Proxy

The [Model Context Protocol (MCP)](https://modelcontextprotocol.io/introduction) is an open protocol that defines a standardized way to provide context to large language models (LLMs). By using MCP, applications can seamlessly interact with LLMs in a structured and efficient manner, making it easier to exchange information, use external tools, and enhance the capabilities of language models in a consistent and reusable way.

This guide will walk you through how to create **MCP clients** that connect to MCP servers, enabling you to utilize external tools and processes efficiently.

## Introduction to MCP

MCP aims to standardize how applications provide contextual information to LLMs and how LLMs can access external tools. MCP servers expose specific tools that are accessible via different communication protocols, such as [`stdio` (Standard Input/Output)](https://spec.modelcontextprotocol.io/specification/2024-11-05/basic/transports/#stdio) and [`SSE` (Server-Sent Events)](https://spec.modelcontextprotocol.io/specification/2024-11-05/basic/transports/#http-with-sse).

MCP clients are responsible for interacting with these servers, sending requests, and processing responses. By integrating MCP into your application, you can build systems where LLMs can utilize external computational tools in real-time.

In this guide, we will:

- Set up an MCP server that exposes basic mathematical tools (`add` and `multiply`).
- Connect the AG2 framework to this MCP server.
- Communicate using two different transport protocols: [stdio](https://spec.modelcontextprotocol.io/specification/2024-11-05/basic/transports/#stdio) and [SSE](https://spec.modelcontextprotocol.io/specification/2024-11-05/basic/transports/#http-with-sse).

## Installation

To integrate MCP tools into the AG2 framework, install the required dependencies:

```bash
pip install -U ag2[openai,mcp,mcp-proxy-gen]
```

> **Note:** If you have been using `autogen` or `pyautogen`, all you need to do is upgrade it using:  
> ```bash
> pip install -U autogen[openai,mcp,mcp-proxy-gen]
> ```
> or  
> ```bash
> pip install -U pyautogen[openai,mcp,mcp-proxy-gen]
> ```
> as `pyautogen`, `autogen`, and `ag2` are aliases for the same PyPI package.  

## Imports

Before diving into the code, let’s go over the main imports used in this guide:

- `ClientSession`: Manages the client session for connecting to the MCP server, allowing you to send requests and handle responses.
- `StdioServerParameters`: Provides parameters to set up communication with an MCP server over `stdio`.
- `stdio_client`: A utility that helps you connect to an MCP server using the standard input/output protocol.
- `sse_client`: A utility to connect to an MCP server using the `SSE` (Server-Sent Events) protocol.
- `create_toolkit`: A helper function that creates a toolkit, wrapping the tools exposed by the MCP server for easier access.


In [ ]:
from pathlib import Path

from mcp import ClientSession, StdioServerParameters
from mcp.client.stdio import stdio_client
from mcp.client.streamable_http import streamablehttp_client

from autogen import LLMConfig
from autogen.agentchat import AssistantAgent
from autogen.mcp import create_toolkit
from autogen.mcp.mcp_proxy import MCPProxy

In [ ]:
# Only needed for Jupyter notebooks
import nest_asyncio

nest_asyncio.apply()

In [ ]:
import asyncio
import threading
from typing import Literal

import uvicorn
from fastapi import FastAPI
from pydantic import BaseModel


# Define request and response models
class AddRequest(BaseModel):
    a: float
    b: float


class AddResponse(BaseModel):
    result: float
    operation: Literal["addition"]


# Create FastAPI app
app = FastAPI(title="Addition API", version="1.0", description="A simple API to add two numbers.")


@app.post("/add", response_model=AddResponse)
def add_numbers(req: AddRequest):
    result = req.a + req.b
    return AddResponse(result=result, operation="addition")


# Function to run the server
def run_server():
    uvicorn.run(app, host="127.0.0.1", port=8000)


# Start FastAPI server in a separate thread
thread = threading.Thread(target=run_server, daemon=True)
thread.start()

await asyncio.sleep(2)  # Give the server a moment to start

## Generate Proxy MCP Server Code

In [ ]:
import shutil

tmp_path = Path("tmp") / "mcp_math"
shutil.rmtree(tmp_path, ignore_errors=True)
tmp_path.mkdir(parents=True, exist_ok=True)

MCPProxy.create(
    openapi_url="http://127.0.0.1:8000/openapi.json",
    client_source_path=tmp_path,
    servers=[{"url": "http://127.0.0.1:8000/"}],
)

assert tmp_path.exists(), "Failed to create tmp directory"
# assert list(tmp_path.glob("*.py")) >= 2, "Failed to generate OpenAPI client files"

## Creating a Toolkit from MCP Tools

To allow AG2 to interact with the MCP server, we need to establish a connection, create a toolkit from the tools exposed by the server, and then register this toolkit with an AG2 agent. The toolkit will provide the necessary functionality for AG2 to call the MCP tools, such as performing mathematical operations.

**Steps to Create a Toolkit from MCP Tools**

- **Wrap MCP tools into a toolkit**: The tools exposed by the MCP server (like `add` and `multiply`) are wrapped into a toolkit for easy use by AG2.
- **Register the toolkit with an AG2 agent**: This step allows AG2 to use the toolkit’s tools when processing requests.
- **Start a conversation**: AG2 can now send a message to the MCP server requesting a task to be performed (e.g., adding two numbers) and receive the result.

In [ ]:
async def create_toolkit_and_run(session: ClientSession) -> None:
    # Create a toolkit with available MCP tools
    toolkit = await create_toolkit(session=session)
    agent = AssistantAgent(name="assistant", llm_config=LLMConfig(model="gpt-4o-mini", api_type="openai"))
    # Register MCP tools with the agent
    toolkit.register_for_llm(agent)

    # Make a request using the MCP tool
    result = await agent.a_run(
        message="Add 123223 and 456789",
        tools=toolkit.tools,
        max_turns=2,
        user_input=False,
    )

    await result.process()

## Setting Up the Client Session and Communicating with the MCP Server

Once the toolkit is created, the next step is to connect AG2 to the MCP server and manage the communication between them. This involves setting up connection parameters and starting a client session, which will facilitate the entire interaction. After establishing the connection, AG2 can send requests and receive responses from the server.

### Steps to Set Up the Client Session and Communicate with the MCP Server

- **Set up connection parameters**: Define how AG2 will communicate with the MCP server. This includes specifying the transport protocol (`stdio` or `SSE`) and other connection details.
- **Start a client session**: This session facilitates communication between AG2 and the MCP server. The session is responsible for sending requests, receiving responses, and maintaining the connection state.

#### Using `stdio_client` for Communication

Establish the connection in the client code by using the `stdio_client` requires:

- **Server Parameters**: The connection is defined using the `StdioServerParameters`, where the command (`python`) runs the server (`math_server.py`) with `stdio` mode.
- **Connecting to the Server**: The `stdio_client` establishes the connection to the server, and the `ClientSession` is used to facilitate communication.
- **Running the Task**: Once the connection is established, `create_toolkit_and_run(session)` is called to wrap the MCP tools and perform the task asynchronously.

In [ ]:
# Create server parameters for stdio connection

# security = APIKeyHeader.Parameters(value="some_api_key")

server_params = StdioServerParameters(
    # add security parameters here as env var
    command="python",  # The command to run the server
    args=[
        str(tmp_path / "main.py"),  # Path to the server script
        "stdio",
    ],  # Path to server script and transport mode
    # env=security.to_env(),  # Environment variables for security
)

async with stdio_client(server_params) as (read, write), ClientSession(read, write) as session:
    # Initialize the connection
    await session.initialize()
    await create_toolkit_and_run(session)

#### Using `streamablehttp_client` for Communication

Establish the connection in the client code by using the `streamablehttp_client` as follows:

* **Server Parameters**: The server can be launched programmatically by using a helper context manager `run_streamable_http_client`, which starts the `main.py` script with HTTP-based streaming enabled. Configuration such as the port is passed through environment variables (`MCP_SETTINGS`).
* **Connecting to the Server**: The `streamablehttp_client` connects to the running server via an HTTP endpoint (`http://127.0.0.1:8001/mcp`) and sets up streaming read/write interfaces.
* **Running the Task**: Once the connection is active, the `ClientSession` manages the communication flow. The task execution begins with `create_toolkit_and_run(session)` to interact with the MCP tools asynchronously.

In [ ]:
# Start the server using streamable http client

import json

from autogen.mcp.helpers import run_streamable_http_client

async with (
    run_streamable_http_client(
        mcp_server_path=str(tmp_path / "main.py"),
        env_vars={
            "MCP_SETTINGS": json.dumps({
                "port": 8001,
            })
        },
    ),
    streamablehttp_client("http://127.0.0.1:8001/mcp") as (
        read_stream,
        write_stream,
        _,
    ),
    ClientSession(read_stream, write_stream) as session,
):
    await session.initialize()
    await create_toolkit_and_run(session)